# Up and Running with TensorFlow

In [1]:
### SETUP

# For plotting
from mpl_toolkits.mplot3d import Axes3D
import matplotlib
from matplotlib import gridspec
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
custom_cmap = ListedColormap(['#fafab0','#9898ff','#a0faa0'])
%matplotlib inline

# Numpy
import numpy as np

# Scikit learn 
from sklearn.datasets import fetch_california_housing

# For scaling the features
from sklearn.preprocessing import StandardScaler

# TensorFlow
import tensorflow as tf
print(tf.__version__)

/home/igr18/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.10.0


In [2]:
### Creating a computation graph

# Reset graph 
tf.reset_default_graph()

# Create the graph with 2 variables and 
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
# the graph itself
f = x*x*y + y + 2

In [3]:
### Evaluate the graph (1)

# Open a TF session
sess = tf.Session()
# Initialise variables in the session
sess.run(x.initializer)
sess.run(y.initializer)
# Evaluate f in the session
result = sess.run(f)
# Print result
print(result)
# Close session
sess.close()

42


In [4]:
### Evaluate the graph in an easier way (2)

with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    
print(result)

42


A TensorFlow program is typically split into two parts: the first part builds a computation graph (this is called the construction phase), and the second part runs it (this is the execution phase).

In [5]:
### Managing graphs

# Reset graph so it does not end up containing duplicate nodes
tf.reset_default_graph()

# Create variable
x1 = tf.Variable(1)
# Will be in default graph
print(x1.graph is tf.get_default_graph())

# Create new graph 
graph = tf.Graph()
# and temporarily make it the default graph inside a block
with graph.as_default():
    x2 = tf.Variable(2)
    # Will print True because you are inside the with block
    print(x2.graph is tf.get_default_graph())
# Will print True as it is of graph
print(x2.graph is graph)
# Will print False as it is not the default graph
print(x2.graph is tf.get_default_graph())

True
True
True
False


In [6]:
### Lifecycle of a Node Value

# Reset graph 
tf.reset_default_graph()

# Create constant w
w = tf.constant(3)
x = w+2
y = x+5
z = x*3

# Runs them separately, meaning that it evaluates w and x twice
with tf.Session() as sess:
    print(y.eval())
    print(z.eval())
    
# Runs them efficiently in one graph run
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)


10
15
10
15


## Linear Regression with TensorFlow

In [7]:
# Reset graph 
tf.reset_default_graph()

# Fetch the California housing dataset
housing = fetch_california_housing()
# Get the shape of the dataset
m, n = housing.data.shape

# Add an extra bias input feature x_0 = 1 
# to all training instances
# housing has 8 features: MedInc, HouseAge, AveRooms, etc
# after adding it has 9 features
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

# Create 2 TensorFlow constant nodes, X and y
# to hold the data and the targets
# X = 20640 x 9
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
# y = 20640 x 1
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")

# # Define X^T
XT = tf.transpose(X)

# # Define theta = (X^T X)^-1 X^T y
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)

# Create the session and evaluate theta
with tf.Session() as sess: 
    theta_value = theta.eval()
    
print("TF results: \n", theta_value, "\n")

## Compare with pure numpy
X = housing_data_plus_bias
y = housing.target.reshape(-1, 1)
theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

print("NumPy results: \n", theta_numpy, "\n")

## Compare with Scikit learn
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing.data, housing.target.reshape(-1, 1))

print("SciKit learn: \n", np.r_[lin_reg.intercept_.reshape(-1, 1), lin_reg.coef_.T])

TF results: 
 [[-3.7185181e+01]
 [ 4.3633747e-01]
 [ 9.3952334e-03]
 [-1.0711310e-01]
 [ 6.4479220e-01]
 [-4.0338000e-06]
 [-3.7813708e-03]
 [-4.2348403e-01]
 [-4.3721911e-01]] 

NumPy results: 
 [[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]] 

SciKit learn: 
 [[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


## Implementing Gradient Descent

In [8]:
### Scaling the features first
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m,1)), scaled_housing_data]

print(scaled_housing_data_plus_bias.mean(axis=0))
print(scaled_housing_data_plus_bias.mean(axis=1))
print(scaled_housing_data_plus_bias.mean())
print(scaled_housing_data_plus_bias.shape)

[ 1.00000000e+00  6.60969987e-17  5.50808322e-18  6.60969987e-17
 -1.06030602e-16 -1.10161664e-17  3.44255201e-18 -1.07958431e-15
 -8.52651283e-15]
[ 0.38915536  0.36424355  0.5116157  ... -0.06612179 -0.06360587
  0.01359031]
0.11111111111111005
(20640, 9)


In [9]:
### # GD 
# Reset graph 
tf.reset_default_graph()

# Define variables
n_epochs = 1000
learning_rate = 0.01

# Create two TF constant nodes
# to hold the data and the targets
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")

# Create 2 TF variables to store theta and the predictions
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

# Create the error variables
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

# Calculate gradients
# 1. Manually
#gradients = 2/m * tf.matmul(tf.transpose(X), error)
# 2. Using TF's autodiff 
gradients = tf.gradients(mse, [theta])[0]

# Create a node that assigns a new value to a variable.
# In this case it implements: theta^(k+1) = theta^(k) - eta grad
training_op = tf.assign(theta, theta - learning_rate * gradients)

# Initialise global variables
init = tf.global_variables_initializer()

# Run the graph
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch ", epoch, " MSE = ", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()
    
best_theta

Epoch  0  MSE =  11.453259
Epoch  100  MSE =  0.7859895
Epoch  200  MSE =  0.60901916
Epoch  300  MSE =  0.5845958
Epoch  400  MSE =  0.56923044
Epoch  500  MSE =  0.557992
Epoch  600  MSE =  0.54967743
Epoch  700  MSE =  0.543502
Epoch  800  MSE =  0.53889894
Epoch  900  MSE =  0.53545487


array([[ 2.0685523 ],
       [ 0.8592519 ],
       [ 0.15122917],
       [-0.27115855],
       [ 0.28820804],
       [ 0.00692668],
       [-0.04270531],
       [-0.6353016 ],
       [-0.6069699 ]], dtype=float32)

In [10]:
### # GD with optimiser 
# Reset graph 
tf.reset_default_graph()

# Define variables
n_epochs = 1000
learning_rate = 0.01

# Create two TF constant nodes
# to hold the data and the targets
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")

# Create 2 TF variables to store theta and the predictions
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

# Create the error variables
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

# Calculate gradients with optimiser directly
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

# Initialise global variables
init = tf.global_variables_initializer()

# Run the graph
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch ", epoch, " MSE = ", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()
    
best_theta 



Epoch  0  MSE =  9.790122
Epoch  100  MSE =  0.8067699
Epoch  200  MSE =  0.62906545
Epoch  300  MSE =  0.60218865
Epoch  400  MSE =  0.5839549
Epoch  500  MSE =  0.5702584
Epoch  600  MSE =  0.55989265
Epoch  700  MSE =  0.5520095
Epoch  800  MSE =  0.54598475
Epoch  900  MSE =  0.5413569


array([[ 2.0685523 ],
       [ 0.8950173 ],
       [ 0.15788078],
       [-0.33883458],
       [ 0.344216  ],
       [ 0.00888295],
       [-0.04403077],
       [-0.5532375 ],
       [-0.5290735 ]], dtype=float32)

Predictions:

In [11]:
Yreal = housing.target.reshape(-1, 1)

Ypred = np.transpose(np.matmul(np.transpose(best_theta), np.transpose(scaled_housing_data_plus_bias)))
np.mean((Ypred - Yreal)**2)

0.5377837375479811

## Feeding Data to the Training Algorithm

To do mini-batch GD:
> We need a way to replace X and y at every iteration with the next mini-batch. 
The simplest way to do this is to use ```placeholder``` nodes. 
These nodes are special because they don’t actually perform any computation, they just output the data you tell them to output at runtime.
They are typically used to pass the training data to TensorFlow
during training. If you don’t specify a value at runtime for a placeholder, you get an
exception.

In [12]:
# Placeholder nodes are used as input nodes 
# (just to output data, not evaluate it)
A = tf.placeholder(tf.float32, shape = (None,3))
B = A + 5

with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1,2,3]]})
    B_val_2 = B.eval(feed_dict={A: [[4,5,6],[7,8,9]]})
    
print(B_val_1)    
print(B_val_2) 

n,m

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


(8, 20640)

In [13]:
### # Mini-batch GD with placeholder nodes for the mini-batches
# Reset graph 
tf.reset_default_graph()

# Define variables
n_epochs = 10
learning_rate = 0.01
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

# Create two TF placeholder nodes
# to hold the data and the targets
X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(tf.float32, shape=(None,   1), name="y")

# Create 2 TF variables to store theta and the predictions
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

# Create the error variables
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

# Calculate gradients with optimiser directly
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

# Initialise global variables
init = tf.global_variables_initializer()

# Function to fetch the mini-batch data
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)    # not shown in the book
    indices = np.random.randint(m, size = batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices]   # not shown
    y_batch = housing.target.reshape(-1, 1)[indices]   # not shown
    return X_batch, y_batch


# Run the graph
with tf.Session() as sess:
    # Initialise
    sess.run(init)
    
    # Epochs
    for epoch in range(n_epochs):
        # Batches
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    # Get best theta
    best_theta = theta.eval()
    
best_theta 

array([[ 2.0702524 ],
       [ 0.85913193],
       [ 0.12432421],
       [-0.29847068],
       [ 0.3707138 ],
       [ 0.00520591],
       [-0.0130143 ],
       [-0.82407236],
       [-0.79245   ]], dtype=float32)

## Visualizing the Graph and Training Curves with TensorBoard

In [14]:
### # Using TensorBoard
# Reset graph 
tf.reset_default_graph()

# Add a time stamp to the log files, otherwise TB merges stats
from datetime import datetime

now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

# Define variables
n_epochs = 50
batch_size = 100
n_batches = int(np.ceil(m / batch_size))
learning_rate = 0.01

# Create two TF placeholder nodes
# to hold the data and the targets
X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

# Create 2 TF variables to store theta and the predictions
theta  = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

# Create the error variables
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

# Calculate gradients with optimiser directly
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

# Initialise global variables
init = tf.global_variables_initializer()

### Store stats:
# Creates a node in the graph 
# that will evaluate the MSE value and write it
# to a TensorBoard-compatible binary log string called a 'summary'
mse_summary = tf.summary.scalar('MSE', mse)
# Creates a FileWriter that you will use to write 
# summaries to logfiles in the log directory
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

# Function to fetch the mini-batch data
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)    # not shown in the book
    indices = np.random.randint(m, size = batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices]   # not shown
    y_batch = housing.target.reshape(-1, 1)[indices]   # not shown
    return X_batch, y_batch


# Run the graph
with tf.Session() as sess:
    # Initialise
    sess.run(init)
    
    # Epochs
    for epoch in range(n_epochs):
        # Batches
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            
            ### Store stats about the MSE every 10 mini-batches
            if batch_index % 10 == 0:
                # Fetch the summary
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            
            # Run for next
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
    # Get best theta
    best_theta = theta.eval()
    
# Close the writer
file_writer.close()

best_theta 

array([[ 2.0552726 ],
       [ 0.82219887],
       [ 0.11761035],
       [-0.25219893],
       [ 0.32396972],
       [ 0.01295227],
       [-0.04279889],
       [-0.9030713 ],
       [-0.87828976]], dtype=float32)

## Names Scopes

In [15]:
### # Using name scopes to group related nodes
# Reset graph 
tf.reset_default_graph()

# Add a time stamp to the log files, otherwise TB merges stats
from datetime import datetime

now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

# Define variables
n_epochs = 50
batch_size = 100
n_batches = int(np.ceil(m / batch_size))
learning_rate = 0.01

# Create two TF placeholder nodes
# to hold the data and the targets
X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

# Create 2 TF variables to store theta and the predictions
theta  = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

### Create the name scope here
with tf.name_scope("loss") as scope:
    # Create the error variables
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")

# Calculate gradients with optimiser directly
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

# Initialise global variables
init = tf.global_variables_initializer()

### Store stats:
# Creates a node in the graph 
# that will evaluate the MSE value and write it
# to a TensorBoard-compatible binary log string called a 'summary'
mse_summary = tf.summary.scalar('MSE', mse)
# Creates a FileWriter that you will use to write 
# summaries to logfiles in the log directory
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

# Function to fetch the mini-batch data
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)    # not shown in the book
    indices = np.random.randint(m, size = batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices]   # not shown
    y_batch = housing.target.reshape(-1, 1)[indices]   # not shown
    return X_batch, y_batch


# Run the graph
with tf.Session() as sess:
    # Initialise
    sess.run(init)
    
    # Epochs
    for epoch in range(n_epochs):
        # Batches
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            
            ### Store stats about the MSE every 10 mini-batches
            if batch_index % 10 == 0:
                # Fetch the summary
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            
            # Run for next
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
    # Get best theta
    best_theta = theta.eval()
    
# Close the writer
file_writer.flush()
file_writer.close()

print("Best theta:")
print(best_theta)

Best theta:
[[ 2.0552726 ]
 [ 0.82219887]
 [ 0.11761035]
 [-0.25219893]
 [ 0.32396972]
 [ 0.01295227]
 [-0.04279889]
 [-0.9030713 ]
 [-0.87828976]]
